In [42]:
# Import library
import requests
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import base64
from requests.exceptions import ReadTimeout
import csv
import datetime

In [43]:
# IDs Niccolò (ids no longer active)
#client_id = "dcbca36048b643a2927249b8c5cf66d9"
#client_secret = "da5fe7ed05a0487a964d25eef7592d6c"

In [44]:
headers = {}
client_creds = f"{client_id}:{client_secret}"
client_creds_base64 = base64.b64encode(client_creds.encode())
token_url = 'https://accounts.spotify.com/api/token'
token_data = {"grant_type": "client_credentials"}
token_headers = {"Authorization":f"Basic {client_creds_base64.decode()}" }

def get_new_token():
    r = requests.post(token_url, headers=token_headers, data=token_data)
    valid_request = r.status_code in range(200, 299)

    if valid_request:
        r = r.json()
        access_token = r['access_token']
        expires_in = r["expires_in"]    # in second
        # print(access_token, expires_in)

        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds=expires_in)
        # did_expire = expires < now

        # Construct Spotify API query
        headers = {
            'Authorization': 'Bearer '+ access_token,
            'Content-type': 'application/json',
        }
    return expires, headers

expires, headers = get_new_token()

In [45]:
track_url = 'https://api.spotify.com/v1/recommendations/available-genre-seeds'

# Query Spotify API
try:
    if expires < datetime.datetime.now(): 
        print("Get new Token")
        expires, headers = get_new_token()
    genres = requests.get(track_url, headers=headers).json()
    # print(json.dumps(genres, indent=4))
except Exception as e: print("\n", e)


# Create a json file for genres list
with open("genres.json", 'w') as f:
   json.dump(genres, f, indent=4)

In [41]:
min_number_of_tracks = 1000
max_null_iteration = 10
stored_tracks = {} # {track_id: genre}
old_tot_track = 0
for genre in tqdm(genres["genres"]):
    number_of_tracks = 0
    null_iteration_count = 0

    track_url = 'https://api.spotify.com/v1/recommendations?' + 'seed_genres=' + genre + "&limit=100"

    while number_of_tracks < min_number_of_tracks and null_iteration_count < max_null_iteration:
        try:
            if expires < datetime.datetime.now(): 
                print("Get new Token")
                expires, headers = get_new_token()
            # tracks_list = sp.recommendations(seed_genres=[genre], limit=100)
            tracks_list = requests.get(track_url, headers=headers).json()                 
        except Exception as e: print("\n", e)
        
        null_iteration = True
        
        if "tracks" in tracks_list:
            for i, track in enumerate(tracks_list["tracks"]):
                if track["id"] not in stored_tracks:
                    stored_tracks[track["id"]] = genre
                    number_of_tracks += 1
                    null_iteration = False

        if null_iteration:
            null_iteration_count += 1
            
    print(genre, len(stored_tracks)-old_tot_track)
    old_tot_track = len(stored_tracks)
    # Create a json file for genres list
    with open("stored_tracks.json", 'w') as f:
       json.dump(stored_tracks, f, indent=4)
    
        

  0%|          | 0/126 [03:41<?, ?it/s]


KeyboardInterrupt: 

In [28]:
# Create a json file for genres list
with open("stored_tracks.json", 'w') as f:
   json.dump(stored_tracks, f, indent=4)

In [29]:
columns = ["id", "track_name", "track_explicit",  "track_popularity", "album_name", "album_release_date", "album_release_date_precision",
           "artist_name", "audio_avg_pitches", "audio_avg_timbre",
           "audio_acousticness", "audio_danceability", "audio_duration_ms", "audio_energy", "audio_instrumentalness",
           "audio_key_1", "audio_liveness", "audio_loudness", "audio_mode_1", "audio_speechiness", "audio_tempo",
           "audio_time_signature", "audio_valence", "track_uri", "track_genre"]

with open('dataframe.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(columns)
    #write.writerows(list_of_track_audio_features)
    f.close()

In [1]:
list_of_track_audio_features = []

with open('stored_tracks.json') as f:
    stored_tracks = json.load(f)

for i,track in enumerate(tqdm(stored_tracks)):

    if expires < datetime.datetime.now(): 
        print("Get new Token")
        expires, headers = get_new_token()
   
            
    try:
      track_info_url = "https://api.spotify.com/v1/tracks/" + track
      track_info = requests.get(track_info_url, headers=headers)
      track_audio_analysis_url = "https://api.spotify.com/v1/audio-analysis/" + track
      track_audio_analysis = requests.get(track_audio_analysis_url, headers=headers)
      features_url = "https://api.spotify.com/v1/audio-features/" + track
      features = requests.get(features_url, headers=headers)  

      valid_request_1 = track_info.status_code in range(200, 299)
      valid_request_2 = track_audio_analysis.status_code in range(200, 299)
      valid_request_3 = features.status_code in range(200, 299)

      if (valid_request_1 and valid_request_2 and valid_request_3):
        track_info = track_info.json()
        track_audio_analysis = track_audio_analysis.json()
        features = features.json()
      else:
        print("\n STATUS:", track_info.status_code, track_audio_analysis.status_code, features.status_code)
        continue
    except Exception as e:
      print(e)
      continue

    track_name = track_info["name"]     
    track_explicit = track_info['explicit']
    track_popularity = track_info["popularity"]
    album_info = track_info['album']
    album_name = album_info['name']
    album_release_date = album_info['release_date']
    album_release_date_precision = album_info['release_date_precision']
    artist_info = track_info['artists'][0]
    artist_name = artist_info['name']
    track_info = [track_name, track_explicit, track_popularity,
                  album_name, album_release_date, album_release_date_precision,
                  artist_name]
  
    #Add in some Spotify Audio Analysis Vectors
    num_seg = 0
    pitches = np.zeros(12)
    timbre = np.zeros(12)   
    if "segments" in track_audio_analysis:
        for _, j in enumerate(track_audio_analysis['segments']):
            pitches += np.array(j['pitches'])
            timbre += np.array(j['timbre'])
            num_seg+=1
    track_features_avg_pitches = list(pitches/num_seg)
    track_features_avg_timbre = list(timbre/num_seg)
    track_audio_analysis_list = [track_features_avg_pitches, track_features_avg_timbre]
  
    # Get audio features for this specific track
    features_list = [features['acousticness'], features['danceability'], features['duration_ms'],
                      features['energy'], features['instrumentalness'], features['key'], 
                      features['liveness'], features['loudness'], features['mode'],  
                      features['speechiness'], features['tempo'], features['time_signature'],
                      features['valence'], features['uri']]
    
    trackData = [track] + track_info + track_audio_analysis_list + features_list + [stored_tracks[track]]
    list_of_track_audio_features.append(trackData)

    with open('dataframe.csv', 'a+', newline='') as f:
    # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerow(trackData)
        f.close()

NameError: name 'json' is not defined

In [ ]:
spotify_track = pd.DataFrame(list_of_track_audio_features, columns = columns)
spotify_track.head()

,id,track_name,track_explicit,track_popularity,album_name,album_release_date,album_release_date_precision,artist_name,audio_avg_pitches,audio_avg_timbre,...,audio_key_1,audio_liveness,audio_loudness,audio_mode_1,audio_speechiness,audio_tempo,audio_time_signature,audio_valence,track_uri,track_genre
0,33DxDFhE06bRC5tdHkIxhj,Gårdakvarnar och skit,False,46,Ett kolikbarns bekännelser,2005-02-09,day,Håkan Hellström,"[0.23305830039525718, 0.4459733201581028, 0.28...","[49.13936462450591, -5.627895256916996, 13.833...",...,9,0.1480,-7.679,1,0.0268,153.146,4,0.547,spotify:track:33DxDFhE06bRC5tdHkIxhj,acoustic
1,5oCMnpuRd3rUL9clNWbjr0,My Hero - Live,False,0,Skin And Bones,2006-11-07,day,Foo Fighters,"[0.3427162592986181, 0.4581859723698199, 0.216...","[49.027059511158434, 82.89144845908609, 21.248...",...,4,0.9600,-6.162,1,0.0566,134.414,4,0.184,spotify:track:5oCMnpuRd3rUL9clNWbjr0,acoustic
2,4I2GqMe7L2ccMpUbnDzYLH,Somebody To Love,False,0,My Worlds (International Version),2010-01-01,day,Justin Bieber,"[0.6901727941176465, 0.6852941176470582, 0.271...","[50.78460416666674, 53.708957107843084, 28.101...",...,5,0.2280,-5.235,1,0.0257,129.970,4,0.636,spotify:track:4I2GqMe7L2ccMpUbnDzYLH,acoustic
3,5sM2GI0vMNZL7tvfPQfByV,Guaranteed,False,0,Music For The Motion Picture Into The Wild,2007-12-17,day,Eddie Vedder,"[0.25286486486486465, 0.16291891891891916, 0.4...","[43.747290322580646, -54.88556059285093, -25.7...",...,7,0.1060,-12.257,1,0.0416,100.466,4,0.127,spotify:track:5sM2GI0vMNZL7tvfPQfByV,acoustic
4,4eiWWDbWczW2GLqg6Lyg48,Cups,False,35,Cups,2012-11-26,day,Lulu & The Lampshades,"[0.4856436781609195, 0.5352547892720306, 0.406...","[46.530522988505744, 36.353842911877365, 23.16...",...,6,0.0675,-6.138,1,0.2280,128.087,4,0.767,spotify:track:4eiWWDbWczW2GLqg6Lyg48,acoustic
